## Notebook that will evaluate the KNN model

In [1]:
#importing libraires to use
import sklearn.preprocessing as skl_pre
import sklearn.neighbors as skl_nb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#need to get the processing python file in another directory
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from process_data import process_data

In [ ]:
#getting the percentage splits of our training and testing data
split_prec = {
    'train': 0.7, 
    'valid': 0.3,
}
#whatever scaler we will use
scaler = skl_pre.MinMaxScaler()
scaler = None

#get the training and testing data
X_train, X_test, Y_train, Y_test = process_data(split_prec, scaler,is_random=False)
print(X_train)
# print(X_test)

Dropped columns: ['snow']
New columns: ['good_weather' 'is_day' 'temp_fahrenheit']
      hour_of_day  day_of_week  month  holiday  weekday  summertime  temp  \
0               5            5      1        0        0           0  -7.2   
1              21            4      1        0        1           0  -1.3   
2              21            3      8        0        1           1  26.9   
3               1            6      1        0        0           0   3.1   
4              17            0      3        0        1           0  11.7   
...           ...          ...    ...      ...      ...         ...   ...   
1595            3            5      6        0        0           1  21.5   
1596           14            0      6        0        1           1  23.2   
1597           13            0      3        0        1           1  13.9   
1598           14            5      3        0        0           1  11.7   
1599           22            6      2        0        0           0   

### Training  
Now that we have our training and test data we will run a KNN model on the training data with k-fold validation for hyperparameter tunning (what is a good value of k, distance measure to use, ect...). Then once we have a decent value of k we will "retrain" the model on the entire training data set and use that model on the test data which has never been seen and use the error from that as an estimation of $E_{new}$

In [3]:
#returns a trained KNN classifier with the specifications passed
def get_fitted_KNN(X,Y, k = 1, met = "minkowski"):
    classifier = skl_nb.KNeighborsClassifier(n_neighbors=k, metric=met).fit(X,Y)
    return classifier